In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### Pizza or Not Pizza:

Using Inception V3 trained on imagenet, and the Kaggle dataset "Pizza or Not Pizza": https://www.kaggle.com/datasets/carlosrunner/pizza-not-pizza, I made a binary classifier that takes in an image and reports whether the image is of pizza or not. The data is roughly 2000 images, half of pizza and half of other foods. The data is images of different sizes, so I do some preprocessing to resize all images to 512x512 pixels. I create training, validation, and test data generators, and feed these into the model. Initially, the model was trained on just the softmax and dense layer added on top of Inception. I then trained again after unfreezing the top two layers of Inception. These models are "PizzaOne" and "PizzaTwo" respectively. At the end is just some code to take in a new image and apply the model. 

In [2]:
data_dir = '/Users/ryanflynn/Desktop/pizzaNotPizza/'
input_shape = (512, 512, 3)

In [3]:
## generate data using imageDataGenerator, will pull from train/validation/test folders
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Createtrain, validation and test generators
batch_size = 32
train_generator = datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

valid_generator = datagen.flow_from_directory(
    os.path.join(data_dir, 'valid'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1680 images belonging to 2 classes.
Found 204 images belonging to 2 classes.
Found 82 images belonging to 2 classes.


In [4]:
# Load the InceptionV3 model, chop off the top layer
base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape)

## add pooling and dense fully connected layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Add a final dense layer with softmax activation for the number of classes (should be 2)
n_classes = len(train_generator.class_indices)
predictions = Dense(n_classes, activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# Train the model, now with early stoppping and the top two layers set to trainable
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True
    
# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
## PizzaTwo
early = EarlyStopping(monitor='val_loss', patience=2)
checkpoint = ModelCheckpoint('PizzaTwo.h5', monitor='val_loss', save_best_only=True)

epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator,
    callbacks=[early, checkpoint])

Epoch 1/10
53/53 [==============================] - 889s 17s/step - loss: 0.2762 - accuracy: 0.9024 - val_loss: 1.2968 - val_accuracy: 0.9265
Epoch 2/10
53/53 [==============================] - 851s 16s/step - loss: 0.0879 - accuracy: 0.9690 - val_loss: 0.4819 - val_accuracy: 0.9265
Epoch 3/10
53/53 [==============================] - 840s 16s/step - loss: 0.0597 - accuracy: 0.9839 - val_loss: 0.3450 - val_accuracy: 0.9265
Epoch 4/10
53/53 [==============================] - 839s 16s/step - loss: 0.0520 - accuracy: 0.9792 - val_loss: 0.4141 - val_accuracy: 0.9167
Epoch 5/10
53/53 [==============================] - 917s 17s/step - loss: 0.0478 - accuracy: 0.9857 - val_loss: 0.3360 - val_accuracy: 0.9265
Epoch 6/10
53/53 [==============================] - 844s 16s/step - loss: 0.0273 - accuracy: 0.9911 - val_loss: 0.4614 - val_accuracy: 0.9363
Epoch 7/10
53/53 [==============================] - 822s 15s/step - loss: 0.0351 - accuracy: 0.9899 - val_loss: 0.2928 - val_accuracy: 0.9216
Epoch 

In [16]:
# Train the model for a number of epochs (PizzaOne)
# Train only the top layers
for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator)

Epoch 1/10
53/53 [==============================] - 618s 12s/step - loss: 0.4364 - accuracy: 0.8470 - val_loss: 0.3261 - val_accuracy: 0.8873
Epoch 2/10
53/53 [==============================] - 607s 11s/step - loss: 0.1636 - accuracy: 0.9399 - val_loss: 0.2340 - val_accuracy: 0.9314
Epoch 3/10
53/53 [==============================] - 604s 11s/step - loss: 0.1188 - accuracy: 0.9571 - val_loss: 0.1853 - val_accuracy: 0.9363
Epoch 4/10
53/53 [==============================] - 608s 11s/step - loss: 0.1053 - accuracy: 0.9583 - val_loss: 0.1634 - val_accuracy: 0.9363
Epoch 5/10
53/53 [==============================] - 609s 11s/step - loss: 0.0960 - accuracy: 0.9607 - val_loss: 0.2123 - val_accuracy: 0.9265
Epoch 6/10
53/53 [==============================] - 635s 12s/step - loss: 0.0832 - accuracy: 0.9756 - val_loss: 0.2020 - val_accuracy: 0.9461
Epoch 7/10
53/53 [==============================] - 610s 11s/step - loss: 0.0918 - accuracy: 0.9631 - val_loss: 0.1827 - val_accuracy: 0.9608
Epoch 

In [17]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

3/3 [==============================] - 31s 8s/step - loss: 0.1303 - accuracy: 0.9512
Test accuracy: 0.9512194991111755


In [19]:
model.save('pizzaOne.h5')

### Application

Some code to take any new image and format it correctly to run the model on it. 

In [24]:
model = load_model('pizzaTwo.h5')

In [25]:
imgName = 'calzone.jpg'

In [26]:
def preprocess_image(img):
    img = Image.open(img)
    img = img.resize((512, 512))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)    
    return img_array

In [27]:
newImg = preprocess_image(imgName)
probs = model.predict(newImg)
np.argmax(probs)

1/1 [==============================] - 2s 2s/step


0